# Building Predictive Models

In [1]:
import os
import numpy as np
import pandas as pd

In [4]:
# reading processed data
processed_data_path = os.path.join(os.path.pardir, 'src', 'data', 'processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
test_file_path = os.path.join(processed_data_path, 'test.csv')
train_df = pd.read_csv(train_file_path, index_col='PassengerId')
test_df = pd.read_csv(test_file_path, index_col='PassengerId')

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Survived            891 non-null    int64  
 1   Age                 891 non-null    float64
 2   Fare                891 non-null    float64
 3   FamilySize          891 non-null    int64  
 4   IsMother            891 non-null    int64  
 5   IsMale              891 non-null    int64  
 6   Deck_A              891 non-null    int64  
 7   Deck_B              891 non-null    int64  
 8   Deck_C              891 non-null    int64  
 9   Deck_D              891 non-null    int64  
 10  Deck_E              891 non-null    int64  
 11  Deck_F              891 non-null    int64  
 12  Deck_G              891 non-null    int64  
 13  Deck_Z              891 non-null    int64  
 14  Pclass_1            891 non-null    int64  
 15  Pclass_2            891 non-null    int64  
 16  Pclass_3

In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 418 non-null    float64
 1   Fare                418 non-null    float64
 2   FamilySize          418 non-null    int64  
 3   IsMother            418 non-null    int64  
 4   IsMale              418 non-null    int64  
 5   Deck_A              418 non-null    int64  
 6   Deck_B              418 non-null    int64  
 7   Deck_C              418 non-null    int64  
 8   Deck_D              418 non-null    int64  
 9   Deck_E              418 non-null    int64  
 10  Deck_F              418 non-null    int64  
 11  Deck_G              418 non-null    int64  
 12  Deck_Z              418 non-null    int64  
 13  Pclass_1            418 non-null    int64  
 14  Pclass_2            418 non-null    int64  
 15  Pclass_3            418 non-null    int64  
 16  Title

In [ ]:
# merge both dfs
# df = pd.concat((train_df, test_df), axis=0)

## Data Preperation

In [13]:
X = train_df.loc[:,'Age':].to_numpy().astype('float')
y = train_df['Survived'].ravel()

In [14]:
X.shape, y.shape

((891, 32), (891,))

In [15]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(712, 32) (712,)
(179, 32) (179,)


In [16]:
# average survival in train n test
print('mean survival in train: {0:.3f}'.format(np.mean(y_train)))
print('mean survival in test: {0:.3f}'.format(np.mean(y_test)))

mean survival in train: 0.383
mean survival in test: 0.385


### Check Scikit-Learn version

In [17]:
import sklearn

In [18]:
sklearn.__version__

'0.23.2'

### Baseline Model

In [20]:
# import function
from sklearn.dummy import DummyClassifier

In [21]:
# create model
model_dummy = DummyClassifier(strategy='most_frequent', random_state=0)

In [22]:
# train model
model_dummy.fit(X_train, y_train)

DummyClassifier(random_state=0, strategy='most_frequent')

In [23]:
print('score for baseline model: {0:.2f}'.format(model_dummy.score(X_test, y_test)))

score for baseline model: 0.61


In [24]:
 # performance matrix
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [26]:
# accuracy score
print('accuracy for baseline model: {0:.2f}'.format(accuracy_score(y_test, model_dummy.predict(X_test))))

accuracy for baseline model: 0.61


In [27]:
# confusion matrix
print('confusion matrix for baseline model: \n {0}'.format(confusion_matrix(y_test, model_dummy.predict(X_test))))

confusion matrix for baseline model: 
 [[110   0]
 [ 69   0]]


In [28]:
# precision and recall scores
print('precision for baseline model: {0:.2f}'.format(precision_score(y_test, model_dummy.predict(X_test))))
print('recall for baseline model: {0:.2f}'.format(recall_score(y_test, model_dummy.predict(X_test))))

precision for baseline model: 0.00
recall for baseline model: 0.00


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### First Kaggle submission

In [31]:
# converting test df to a matrix
test_X = test_df.to_numpy().astype('float')

In [32]:
# make predictions
predictions = model_dummy.predict(test_X)

In [33]:
# create teh submission schema
df_submission = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})

In [34]:
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [38]:
submission_data_path = os.path.join(os.path.pardir, 'src', 'data', 'external')
submission_file_path = os.path.join(submission_data_path, 'baseline_model.csv')

In [39]:
df_submission.to_csv(submission_file_path, index=False)

In [54]:
def get_submission_file(model, filename):
    # converting to the matrix
    test_X = test_df.to_numpy().astype('float')
    # make predictions
    predictions = model.predict(test_X)
    # submission dataframe
    df_submission = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})
    # submission file
    submission_data_path = os.path.join(os.path.pardir, 'src', 'data', 'external')
    submission_file_path = os.path.join(submission_data_path, filename)
    # write to file
    df_submission.to_csv(submission_file_path, index=False)

In [ ]:
# get submission file
get_submission_file(model_dummy, 'baseline_model.csv')

## Logistic Regression Model

In [40]:
# import logit model
from sklearn.linear_model import LogisticRegression

In [47]:
# create model
model_logit = LogisticRegression(random_state=0)

In [48]:
# train model
model_logit.fit(X_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [49]:
# evaluate model
print('score for logistic regression - version 1: {0:.2f}'.format(model_logit.score(X_test, y_test)) )

score for logistic regression - version 1: 0.83


In [50]:
# performance matrix
# accuracy score
print('accuracy for logistic regression model: {0:.2f}'.format(accuracy_score(y_test, model_logit.predict(X_test))))
# confusion matrix
print('confusion matrix for logistic regression model: \n {0}'.format(confusion_matrix(y_test, model_logit.predict(X_test))))
# precision and recall scores
print('precision for logistic regression model: {0:.2f}'.format(precision_score(y_test, model_logit.predict(X_test))))
print('recall for logistic regression model: {0:.2f}'.format(recall_score(y_test, model_logit.predict(X_test))))

accuracy for logistic regression model: 0.83
confusion matrix for logistic regression model: 
 [[95 15]
 [15 54]]
precision for logistic regression model: 0.78
recall for logistic regression model: 0.78


In [52]:
# model coefficients
model_logit.coef_

array([[-0.02957907,  0.00414612, -0.48810284,  0.55877059, -0.76106792,
         0.12133613, -0.07415307, -0.35962565,  0.47359246,  1.01789061,
         0.26479044, -0.04874881, -0.36228257,  0.90659602,  0.4916518 ,
        -0.36544827,  0.1080444 ,  1.03868053,  0.52133449, -1.49056376,
         1.16448858, -0.10039552, -0.20878918,  0.11933636,  0.2234472 ,
         0.26959053,  0.42042546,  0.44316842,  0.47305422,  0.11657692,
         0.33832464,  0.6944749 ]])

In [56]:
# make 2nd submission for improved model
get_submission_file(model_logit, 'logit_model.csv')

## Part- 2

### Hyperparameter Optimization

In [57]:
from sklearn.model_selection import GridSearchCV

In [60]:
parameters = {'C':[1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty': ['l1', 'l2']}
clf = GridSearchCV(model_logit, param_grid=parameters, cv=3)

In [61]:
clf.fit(X_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logi

GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']})

In [62]:
clf.best_params_

{'C': 10.0, 'penalty': 'l2'}

In [63]:
print('best score : {0:.2f}'.format(clf.best_score_))

best score : 0.83


In [64]:
# evaluate model
print('score for logistic regression - version 2: {0:.2f}'.format(clf.score(X_test, y_test)))

score for logistic regression - version 2: 0.84


## Feature Normalization & Standardization

In [65]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

**Feature Normalization:** normalizing the data on a same scale

In [66]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [67]:
# extracting one column to check its min & max value to verify if normalization worked
X_train_scaled[:,0].min(), X_train_scaled[:,0].max()

(0.0, 1.0)

In [69]:
# normalizing test data
X_test_scaled = scaler.transform(X_test)

**Feature Standardization:** standardizating the data distribution by fitting it with a distribution of mean 0 and std dev 1.

In [71]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Create model after Normalization & Standardization

In [73]:
# create model
model_logit_v2 = LogisticRegression(random_state=0)
parameters = {'C':[1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty': ['l1', 'l2']}
clf = GridSearchCV(model_logit_v2, param_grid=parameters, cv=3)
clf.fit(X_train_scaled, y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_v

GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']})

In [76]:
clf.best_score_

0.8089978607476747

In [78]:
clf.best_params_

{'C': 1.0, 'penalty': 'l2'}

In [77]:
# evaluate model
print('score for logistic regression - version 2: {0:.2f}'.format(clf.score(X_test_scaled, y_test)))

score for logistic regression - version 2: 0.84


## Model Persistence

In [79]:
# import pickle library
import pickle

In [83]:
# create the file paths
model_file_path = os.path.join(os.path.pardir, 'models', 'model_logit.pkl')
scaler_file_path = os.path.join(os.path.pardir, 'models', 'scaler_logit.pkl')

<_io.BufferedWriter name='../models/scaler_logit.pkl'>


In [81]:
# open the files to write
model_file_pickle = open(model_file_path, 'wb')
scaler_file_pickle = open(scaler_file_path, 'wb')

In [82]:
# persist the model and scaler
pickle.dump(clf, model_file_pickle)
pickle.dump(scaler, scaler_file_pickle)

In [84]:
# close the file
model_file_pickle.close()
scaler_file_pickle.close()

### Load the persisted files

In [86]:
# open the files in read mode
model_file_pickle = open(model_file_path, 'rb')
scaler_file_pickle = open(scaler_file_path, 'rb')
# load files
clf_loaded = pickle.load(model_file_pickle)
scaler_loaded = pickle.load(scaler_file_pickle)
# close files
model_file_pickle.close()
scaler_file_pickle.close()

In [87]:
clf_loaded

GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']})

In [88]:
scaler_loaded

StandardScaler()

In [89]:
# transform the test data using loaded scaler object
X_test_scaled = scaler_loaded.transform(X_test)
# calculate the score using loaded model object
print('score for persisted logistic regression: {0:.2f}'.format(clf_loaded.score(X_test_scaled, y_test)))

score for persisted logistic regression: 0.84
